In [1]:
from bs4 import BeautifulSoup
from pprint import pprint

In [2]:
def find_name(soup):
    name_div = soup.find('div', {"class": "module bio_fighter vcard"})
    fighter_name = soup.find('h1', {'itemprop': 'name'}).span.contents[0]
    return fighter_name

def find_pro_fights(soup, fighter_id):
    fight_divs = soup.findAll('div', {"class": "module fight_history"})
    for section in fight_divs:
        section_type = section.find('div', {"class": "module_header"}).contents[1].text
        if 'Fight History - Pro' == section_type:
            cells = []
            event_date = 'NO FIGHTS FOUND'
            for row in section.findAll('tr')[1:]:
                result_cell, fighter_cell, event_cell, *blah = row.findAll('td')
                result = result_cell.span.contents[0]
                opponent_id_url = fighter_cell.contents[0]['href']
                opponent_id = int(opponent_id_url[opponent_id_url.rfind("-") + 1:])
                event_link = event_cell.contents[0]['href']
                event_date = event_cell.findAll('span')[-1].contents[0]
                fighter_out = (fighter_id, opponent_id, result, event_date, event_link)
                cells += [fighter_out]
                
            return cells
    raise Exception('didnt find any pro fights')

In [ ]:
def parse_fighter_page(page_soup, fighter_id):
    fighter_info = {}
    try:
        fighter_info['fighter_name'] = find_name(page_soup)
        fighter_info['type'] = 'f'
    except AttributeError:
        fighter_info['fighter_name'] = 'no_fighter'
        fighter_info['type'] = 'm'
        return fighter_info
    fighter_info['matches'] = find_pro_fights(page_soup, fighter_id)
    return fighter_info


file_template = 'fighter_lookup{}.html'
for f_id in range(1, 8):
    my_soup = BeautifulSoup(open(file_template.format(f_id), encoding='utf-8'), "html.parser")
    out = parse_fighter_page(my_soup, f_id)
    pprint(out, width=100)
    print('***')
